In [1]:
import pickle
import numpy as np
import tensorflow as tf

# Import Data

In [2]:
# Load the data from the pickle file
with open('../../training_set.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('../../test_set.pkl', 'rb') as f:
    test_data = pickle.load(f)


# Convert the data to a NumPy array 
train_data = np.array(train_data)
test_data = np.array(test_data)

In [3]:
# Look at the data
print(train_data.shape)

(17568, 74)


In [4]:
print(train_data[0])

[12. 11. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 65.]


In [5]:
# Split the data into input and output
# every row is a config
# every column is a value
# the last column is the output
# the first 3 columns are the input
# The first vlaue is the up value
# The second value is the down value
# The third value is the output value

x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]
print(x_train.shape)

(17568, 73)


In [6]:
# Look at first row of input
print(x_train[:1])

[[12. 11. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]]


In [7]:
# Look at first row of output
print(y_train[:1])

[65.]


In [8]:
from keras.layers import Layer
import keras.backend as K

class Round(Layer):

    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)

    def get_output(self, train=True):
        X = self.get_input(train)
        return K.round(X)

    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# Build basic model

In [9]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(73, activation='relu', input_shape=(x_train.shape[1],), name='input'),
    tf.keras.layers.Dense(64, activation='relu', name='hidden'),
    tf.keras.layers.Dense(10, activation='softmax', name='output'),
    # add a layer that rounds the output to the nearest integer
    #tf.keras.layers.Lambda(lambda x: tf.round(x))
])


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-02 13:34:35.879112: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-02 13:34:35.879618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(73, input_dim =73, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(tf.keras.layers.Dense(15, activation = 'relu'))
    model.add(tf.keras.layers.Dense(34, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy'])
    return model

In [11]:
# Create Keras Classifier and use predefined baseline model
estimator = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 64, verbose = 1)
# Try different values for epoch and batch size

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_27353/369795368.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 64, verbose = 1)


In [12]:
# Train the model
estimator.fit(x_train, y_train, epochs=120)

2023-01-02 13:34:36.255003: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/120
  1/275 [..............................] - ETA: 1:55 - loss: 4.0839 - categorical_accuracy: 0.0156

2023-01-02 13:34:36.520208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


275/275 [==============================] - 4s 13ms/step - loss: 2.8638 - categorical_accuracy: 0.1878
Epoch 2/120
275/275 [==============================] - 3s 10ms/step - loss: 1.4455 - categorical_accuracy: 0.6404
Epoch 3/120
275/275 [==============================] - 3s 10ms/step - loss: 0.6552 - categorical_accuracy: 0.8608
Epoch 4/120
275/275 [==============================] - 3s 10ms/step - loss: 0.3435 - categorical_accuracy: 0.8961
Epoch 5/120
275/275 [==============================] - 3s 10ms/step - loss: 0.2494 - categorical_accuracy: 0.9025
Epoch 6/120
275/275 [==============================] - 3s 10ms/step - loss: 0.2167 - categorical_accuracy: 0.9009
Epoch 7/120
275/275 [==============================] - 3s 9ms/step - loss: 0.2022 - categorical_accuracy: 0.9045
Epoch 8/120
275/275 [==============================] - 3s 10ms/step - loss: 0.1961 - categorical_accuracy: 0.9014
Epoch 9/120
275/275 [==============================] - 3s 10ms/step - loss: 0.1921 - categorical_accu

In [14]:
# Save the estimator
#estimator.save('estimator.h5')


AttributeError: 'KerasClassifier' object has no attribute 'save'

In [15]:
# Evaluate the estimator
#estimator.score(x_test, y_test)

 6/69 [=>............................] - ETA: 0s - loss: 0.1682 - categorical_accuracy: 0.9167 

2023-01-02 13:40:13.806237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


69/69 [==============================] - 1s 9ms/step - loss: 0.1816 - categorical_accuracy: 0.8985


ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

In [16]:
predictions = estimator.predict(x_test)

 70/138 [==============>...............] - ETA: 0s

2023-01-02 13:41:05.384171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


138/138 [==============================] - 0s 2ms/step


In [17]:
predictions = estimator.predict(x_test)
sum = 0
for i in range(len(y_test)):
    if predictions[i] == y_test[i]:
        sum += 1
    else:
        print(predictions[i], y_test[i])
accuracy = sum / len(y_test)
print(accuracy)

138/138 [==============================] - 0s 3ms/step
66.0 65.0
68.0 66.0
65.0 66.0
65.0 66.0
68.0 58.0
68.0 66.0
68.0 66.0
68.0 58.0
68.0 66.0
66.0 65.0
68.0 65.0
65.0 66.0
68.0 66.0
66.0 58.0
65.0 66.0
66.0 65.0
68.0 66.0
65.0 66.0
66.0 68.0
66.0 68.0
68.0 58.0
66.0 68.0
66.0 68.0
66.0 68.0
66.0 68.0
68.0 65.0
66.0 58.0
68.0 66.0
68.0 65.0
68.0 66.0
66.0 58.0
66.0 65.0
66.0 68.0
65.0 66.0
68.0 58.0
66.0 65.0
66.0 68.0
65.0 66.0
0.0 1.0
66.0 58.0
68.0 65.0
66.0 68.0
66.0 58.0
66.0 58.0
66.0 68.0
68.0 66.0
66.0 68.0
65.0 66.0
66.0 68.0
68.0 66.0
68.0 65.0
66.0 68.0
66.0 65.0
66.0 68.0
66.0 68.0
66.0 65.0
65.0 66.0
66.0 68.0
66.0 68.0
66.0 68.0
66.0 68.0
66.0 68.0
68.0 65.0
66.0 58.0
66.0 68.0
65.0 66.0
66.0 65.0
66.0 68.0
66.0 68.0
66.0 68.0
65.0 66.0
66.0 65.0
66.0 58.0
68.0 66.0
66.0 68.0
66.0 68.0
65.0 66.0
66.0 68.0
66.0 68.0
66.0 68.0
66.0 58.0
68.0 66.0
66.0 58.0
65.0 66.0
68.0 66.0
68.0 65.0
68.0 58.0
66.0 68.0
68.0 58.0
65.0 66.0
66.0 58.0
68.0 66.0
66.0 68.0
68.0 66.0
65.0 66

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model
model.fit(x_train, y_train, epochs=4, batch_size=32)

# Save the model
model.save('model.h5')

Epoch 1/4


2022-12-21 19:18:58.101736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


549/549 [==============================] - 3s 5ms/step - loss: 1516.0127 - Accuracy: 0.0239
Epoch 2/4
549/549 [==============================] - 3s 5ms/step - loss: 1516.0126 - Accuracy: 0.0192
Epoch 3/4
549/549 [==============================] - 3s 5ms/step - loss: 1516.0114 - Accuracy: 0.0157
Epoch 4/4
549/549 [==============================] - 3s 5ms/step - loss: 1516.0126 - Accuracy: 0.0158


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4736      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 70)                4550      
                                                                 
Total params: 13,446
Trainable params: 13,446
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.evaluate(x_test, y_test)

 24/138 [====>.........................] - ETA: 0s - loss: 1532.9799 - Accuracy: 0.0143

2022-12-21 19:15:04.344002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


138/138 [==============================] - 1s 5ms/step - loss: 1543.8839 - Accuracy: 0.0159


[1543.8839111328125, 0.01593806967139244]

In [ ]:
predictions = model.predict(x_test[1:10])

1/1 [==============================] - 0s 54ms/step


2022-12-21 19:15:08.286251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
print(predictions)

[[0.01436608 0.01463229 0.01376316 0.01421796 0.01378758 0.01428406
  0.01380708 0.01440014 0.01369997 0.01416355 0.01501793 0.01424237
  0.01383106 0.0141349  0.014016   0.01417218 0.01522212 0.01419538
  0.01467267 0.01395234 0.01460596 0.0148426  0.01505581 0.01374258
  0.01432052 0.01495502 0.01384656 0.01370651 0.01430485 0.01438976
  0.01432353 0.01356148 0.01416743 0.0141808  0.0144723  0.01426448
  0.01365536 0.0145434  0.01390234 0.0142574  0.01462263 0.01401307
  0.01524819 0.01455383 0.01384162 0.01434676 0.01391233 0.0145729
  0.01401076 0.01390359 0.01464595 0.01460167 0.01370955 0.01502277
  0.0141082  0.01442356 0.01441614 0.01386539 0.01406827 0.01471683
  0.01359903 0.01470423 0.01456719 0.01492687 0.0143537  0.01338557
  0.01514377 0.01366953 0.01449761 0.014871  ]
 [0.01358154 0.0145574  0.01342274 0.01354806 0.0151034  0.01418611
  0.01424117 0.0142363  0.01422978 0.01449606 0.01492848 0.01445444
  0.01439432 0.01393763 0.01387919 0.01361272 0.01412704 0.01380546
  

In [ ]:
y_test[1:10]

array([47., 26., 32.,  8., 55., 42., 65., 34., 48.])

In [ ]:
# round the predictions
rounded_predictions = np.round(predictions)
# claculate the accuracy
accuracy = np.sum(rounded_predictions == y_test[1:10]) / len(y_test[1:10])
print(accuracy)

0.0


/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_32603/4022209231.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  accuracy = np.sum(rounded_predictions == y_test[1:10]) / len(y_test[1:10])


## Calculate accuracy 

In [ ]:
predictions = model.predict(x_test)
rounded_predictions = np.round(predictions)
sum = 0
for i in range(len(y_test)):
    if rounded_predictions[i] == y_test[i]:
        sum += 1
accuracy = sum / len(y_test)
print(accuracy)

138/138 [==============================] - 0s 2ms/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()